# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [ ]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub

## check bedrock availability

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
from utils.GamePlayer import GamePlayer
from utils.GameAssistant import GameAssistant
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

boto3_bedrock = bedrock.get_bedrock_client()

# print_ww("hello")
# Info("hello")

GA = GameAssistant(template_assistant_role, 1000)
print(GA.agent.prompt.template)

## Test Part

In [ ]:
from collections import Counter

def find_most_frequent(arr):
  freq = Counter(arr)
  max_count = max(freq.values())
  
  return [k for k, v in freq.items() if v == max_count]

vote_names = ['p1','p1','p2','p2','p3','p5','p4','p4']
AA = Counter(vote_names)
print(AA.most_common())

print(find_most_frequent(vote_names))

aa = []

if len(aa) != 1:
    print(1)

In [1]:
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

GM = GameMaster(10)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

02/18/2024 07:27:58 AM 	===== ResetGame P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 =====
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 07:27:58 AM Found credentials from IAM Role: BaseNotebookInstanceEc

 根据你提供的信息,这是游戏的第一晚,我作为狼人P1需要与其他狼人P6达成共识,消灭一个村民。我会建议如下行动:

{"action": "WolfVote", "target": "P2"}

我选择消灭P2。作为狼人,第一晚我需要随机选择一个村民下手,不能暴露任何信息。消灭P2是一个比较安全的选择,不会引起其他人的太多怀疑。希望P6能够理解我的用意,执行这个计划。

02/18/2024 07:28:23 AM 	===== DoAction NIGHT 1 P1 ======
02/18/2024 07:28:23 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '狼人', 'character': '话痨', 'response': ['{"action": "WolfVote", "target": "P2"}']}

02/18/2024 07:28:23 AM 	===== DoMemory NIGHT 1 P6 ======
02/18/2024 07:28:23 AM 	===== DoAnswer NIGHT 1 P6 ======
02/18/2024 07:28:23 AM 	QUESTION: 现在是NIGHT 1,你是玩家P6(狼人身份,本阵营为:P1,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天夜晚，该如何行动?


 我是P6,狼人身份。根据当前的场上情况,第一天晚上我需要与同狼P1进行商讨和配合。我的建议是:

{"action": "WolfVote", "target": "P2"}

我投票消灭P2。作为第一天夜晚,我们需要消灭一个看起来不起眼的普通村民,不引人注意,也不会打草惊蛇。消灭P2是一个稳妥的选择。

02/18/2024 07:28:29 AM 	===== DoAction NIGHT 1 P6 ======
02/18/2024 07:28:29 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P6', 'status': 1, 'role': '狼人', 'character': '过激型', 'response': ['{"action": "WolfVote", "target": "P2"}']}

02/18/2024 07:28:29 AM 	===== PostAction NIGHT 1 ======
02/18/2024 07:28:29 AM 	 [wolf_votes]: ['P2'], [player_vote_name]: Counter({'P2': 2})
02/18/2024 07:28:29 AM 
 [WOLF VOTE]={'time': 'NIGHT 1', 'player': 'P2', 'status': 0, 'role': '村民', 'character': '腼腆型', 'response': '玩家P2于NIGHT 1被狼人投票而出局'}

02/18/2024 07:28:29 AM 	时间NIGHT 1,场上存活状态 狼人:2 村民:5
02/18/2024 07:28:29 AM 游戏继续. 目前玩家存活情况:P1:存活,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活
02/18/2024 07:28:29 AM 	===== PreAction DAY 2 ======
02/18/2024 07:28:29 AM 	===== DoMemory DAY 2 P2 ======
02/18/2024 07:28:29 AM 	ASSISTANT QUESTION : [目前场上信息:P1:存活,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(初始配置为2狼人+6村民, 游戏每轮发言顺序为P1,P2,P3,P4,P5,P6,P7,P8).] 	时间NIGHT 1,场上存活状态 狼人:2 村民:5."[玩家P2于时间NIGHT 1, 执行动作为:玩家P2于NIGHT 1被狼人投

...

02/18/2024 07:54:48 AM 	===== DoAnswer DAY 4 P5 ======
02/18/2024 07:54:48 AM 	QUESTION: 现在是DAY 4,你是玩家P5(村民身份)..目前场上玩家:P1:存活,P2:淘汰,P3:淘汰,P4:淘汰,P5:淘汰,P6:淘汰,P7:存活,P8:淘汰(逗号为分割符).现在是第4天白天，你已经死亡,有何遗言?


 根据你提供的游戏信息,我总结当前游戏状态如下:

现场只剩P1和P7两名存活玩家。我作为P5已被淘汰出局。前几轮死亡的玩家P2、P3、P4、P6、P8均曾提出过对其他玩家的怀疑。

通过这几轮游戏的观察和推理,我判断P6很有可能是狼人。另外,我也认为P1一直比较理性和冷静,值得相信他应该属于村民阵营。

虽然我已无法继续游戏,但为了村民的最终胜利,我选择在死前发表以下遗言:

{"action": "DeathWords", "content": "经过观察,我判断P6极有可能是狼人。同时我也相信P1应该属于我们村民阵营,希望P7能够相信他。大家一定要小心行事,争取最后的胜利。"}

希望这些信息能帮助村民们识破狼人,获得最终的胜利。请继续加油!

02/18/2024 07:55:15 AM 	===== DoAction DAY 4 P5 ======
02/18/2024 07:55:15 AM 
 [ROUND ACTION]={'time': 'DAY 4', 'player': 'P5', 'status': 0, 'role': '村民', 'character': '独立思考', 'response': ['{"action": "DeathWords", "content": "经过观察,我判断P6极有可能是狼人。同时我也相信P1应该属于我们村民阵营,希望P7能够相信他。大家一定要小心行事,争取最后的胜利。"}']}

02/18/2024 07:55:15 AM 	时间DAY 4,场上存活状态 狼人:1 村民:1
02/18/2024 07:55:15 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


 根据文本描述,我总结了以下重点内容:

夜晚1,狼人P1和P6共同投死村民P2。

白天2,P2临终遗言怀疑P4是狼人。P3、P8等人讨论后投票死亡P4,事后P4临终遗言怀疑P3。

夜晚2,狼人P1、P6共同投死村民P3。 

白天3,P3临终遗言怀疑P6是狼人。经讨论后,大多数玩家投票死亡P6。P6临终遗言否认是狼人。

夜晚3,狼人P1单独投死村民P8。

白天4,P8临终遗言怀疑P7是狼人。之后P1投票死亡P5,P5临终遗言怀疑P6是狼人。

根据描述,参与游戏的玩家有P1-P8,其中P1和P6是狼人,P2、P3、P4、P5、P8是村民。前4天游戏中,3名村民被投死,1名狼人P6被投死。第4天结束时,场上还有2名玩家P1(狼人)和P7(村民)。

以上是我根据文本提炼出的主要游戏过程,大概在1000字以内,没有给出任何主观推测,仅客观描述重要事件经过。请检查是否符合要求。

02/18/2024 07:55:44 AM 狼人胜利!
02/18/2024 07:55:44 AM 游戏结束.
02/18/2024 07:55:44 AM ===== EndGame P1:存活,P2:淘汰,P3:淘汰,P4:淘汰,P5:淘汰,P6:淘汰,P7:存活,P8:淘汰 =====
